<a href="https://colab.research.google.com/github/boyerb/Investments/blob/master/Ex07-Alpha_Vantage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Investments: Theory, Fundamental Analysis, and Data Driven Analytics**, Bates, Boyer, and Fletcher

# Examples Chapter 7: The Alpha Vantage API
In this example we illustrate how to load in data using the Alpha vantage API. We then plot the closing price for Coca-Cola (ticker:'KO').

### Imports and Setup
In this example we illustrate how to access data on equities using the Alpha Vantage API. The script uses the function `parse_monthly_alpha_vantage_response()` to formate the data from Alpha Vantage into a noce DataFrame. This function is part of the simple_finance custom module that can be accessed from the course GitHub repo. A module is a single Python file that contains functions, classes, or variables. By packaging code into well-designed functions, this module keeps our Python code clean, helps us avoid getting bogged down in less-important coding details, and maintains focus on core objectives. The complete code for this module can be seen at https://github.com/boyerb/Investments/blob/master/functions/simple_finance.py.

After importing the `simple_finance.py` module we then import the `requests` package for retrieving data and  `pandas` for handling DataFrames. To improve readability when printing a DataFrame to the consol, we then configure Pandas display options: (1) show all columns without truncation, (2) widen the output to 1000 characters so rows print on a single line, and (3) truncate long tables.


In [ ]:
# Import the simple_finance.py package and avoid using a cached version.
# Remove any stale copy of simple_finance.py cached on your PC.
# Grab the newest simple_finance.py file directly from the repo
# Them, force Colab to use the most recent version.
!rm -f simple_finance.py
!curl -O https://raw.githubusercontent.com/boyerb/Investments/master/functions/simple_finance.py
import importlib, simple_finance as sf
importlib.reload(sf)

import requests
import pandas as pd

pd.set_option('display.max_columns', None)   # Show all columns without truncation
pd.set_option('display.width', 1000)   # Set the display width so output stays on one line
pd.set_option("display.max_rows", 20) # Force truncation if DataFrame has more than 20 rows


### Load in Data Using the API
You will first need to get your own API key to run this block of code at this link: https://www.alphavantage.co/support/#api-key.  If you don't supply your own API key, the code will still run, but the number of daily requests will be **limited by IP address**.

When using this block of code in other programs remember to
1. Replace `symbol=KO` in the url string with the symbol for which you want data.
2. Replace `your_key` in the script with your actual key.  

In [ ]:
# Define the API endpoint URL
# The 'symbol' parameter specifies the ticker (KO = Coca-Cola).
# Replace 'your_key' with your personal Alpha Vantage API key.
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY_ADJUSTED&symbol=KO&apikey=your_key'

# Send an HTTP GET request to the API endpoint
r = requests.get(url)
df = sf.parse_monthly_alpha_vantage_response(r)
print(df)

### Plot Closing Price for Coca-Cola
Here we plot the closing price each month for Coca-Cola. As needed, use an AI assitant for further undersatnding of this code.

In [ ]:
df["Close"].plot(figsize=(12,6))

plt.xlabel("Date")
plt.ylabel("Closing Price")
plt.title("Closing Price Over Time")
plt.show()